In [1]:
# Importe
import pandas as pd
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# UFO Datensatz
path = r"C:\Users\Admin\Desktop\07 Python für Datenanalysten\Projekt_UFO\Nadja\ufo_sightings_scrubbed_clean (1).csv"
ufo_df = pd.read_csv(path)

ufo_df.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Admin\\Desktop\\07 Python für Datenanalysten\\Projekt_UFO\\Nadja\\ufo_sightings_scrubbed_clean (1).csv'

In [ ]:
# fehlende Städte suchen und speichern
# Geolokalisierer initialisieren
geolokalisierer = Nominatim(user_agent="geoapiExercises")

# Liste für die Ergebnisse
results = []

# Jede Zeile im DataFrame durchlaufen
for index, row in ufo_df.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    city = row.get('city', '')  # Hier wird angenommen, dass die Stadt in einer Spalte namens 'city' gespeichert ist

    # Überprüfen, ob das Stadtfeld leer ist
    if pd.isna(city) or city.strip() == "":
        # Verwende die Koordinaten, um die Stadt zu finden
        try:
            standort = geolokalisierer.reverse((latitude, longitude))
            if standort:
                # Speichere die Koordinaten, die Stadt und den Status in der Ergebnisliste
                results.append({'latitude': latitude, 'longitude': longitude, 'result_city': standort.address})
            else:
                results.append({'latitude': latitude, 'longitude': longitude, 'result_city': 'Nicht gefunden'})
        except Exception as e:
            results.append({'latitude': latitude, 'longitude': longitude, 'result_city': f'Fehler: {e}'})
    else:
        # Wenn die Stadt bereits eingetragen ist, speichere sie einfach
        results.append({'latitude': latitude, 'longitude': longitude, 'result_city': city})

# Erstelle ein neues DataFrame aus den Ergebnissen
results_df = pd.DataFrame(results)

# DataFrame als CSV speichern
results_df.to_csv(r"C:\Users\Admin\Desktop\07 Python für Datenanalysten\Projekt_UFO\Nadja\ufo_sightings_cities.txt", index=False)


In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import pycountry

# Weltkarte mit Ländergrenzen laden
world = gpd.read_file(r"C:\Users\Patrick\Desktop\Nadja\Nadjas Dateien\shapes_from_natural_earth\ne_10m_admin_0_countries\ne_10m_admin_0_countries.shp")

def get_country_offline(lat, lon):
    point = Point(lon, lat)  # Longitude zuerst in Shapely
    country = world[world.contains(point)]

    return country.iloc[0].NAME if not country.empty else None

# Beispiel
name_of_country = get_country_offline(48.8566, 2.3522)

country_code_dict = {country.name: country.alpha_2 for country in pycountry.countries}

def get_iso_code(country_name):
    return country_code_dict.get(country_name, None)

print(get_iso_code(name_of_country).lower())

fr


In [ ]:
world = gpd.read_file(r"C:\Users\Patrick\Desktop\Nadja\Nadjas Dateien\shapes_from_natural_earth\ne_10m_admin_1_states_provinces\ne_10m_admin_1_states_provinces.shp")

def get_state_offline(lat, lon):
    point = Point(lon, lat)  # Longitude zuerst in Shapely
    state = world[world.contains(point)]

    return state.iloc[0]["name"] if not state.empty else None

# Beispiel
name_of_state = get_state_offline(53.4017, 8.4531)
print(name_of_state)
name_of_state = get_state_offline(28.3922, -80.6077)
print(name_of_state)

Niedersachsen
Florida


In [ ]:
# Zwei Dateien zusammenfügen
import pandas as pd
# CSV-Dateien einlesen
cities_df = pd.read_csv(r"C:\Users\Patrick\Desktop\Nadja\Nadjas Dateien\ufo_sightings_cities.csv")
all_data_df = pd.read_csv(r"C:\Users\Patrick\Desktop\Nadja\Nadjas Dateien\ufo_sightings_scrubbed.csv")

all_data_df.insert(1,"result_city",cities_df["result_city"])
print(all_data_df["country"].isna().sum())
print(all_data_df[all_data_df["city"].isna()])
# Überprüfen der Spaltennamen
cities_df.columns
all_data_df.columns

# Zusammenführen der beiden DataFrames basierend auf den Spalten 'latitude' und 'longitude'
# merged_df = pd.merge(cities_df, all_data_df, left_index=True, right_index=True, how="right")
# print(merged_df)

# Optional: Speichern Sie das zusammengeführte DataFrame in einer neuen CSV-Datei
# merged_df.to_csv('merged_data_ufo_sightings.csv', index=False)

print("Die Daten wurden erfolgreich zusammengeführt und in 'merged_data_ufo_sightings.csv' gespeichert.")

9670
Empty DataFrame
Columns: [datetime, result_city, city, state, country, shape, duration (seconds), duration (hours/min), comments, date posted, latitude, longitude ]
Index: []
Die Daten wurden erfolgreich zusammengeführt und in 'merged_data_ufo_sightings.csv' gespeichert.


C:\Users\Patrick\AppData\Local\Temp\ipykernel_41160\12020029.py:5: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data_df = pd.read_csv(r"C:\Users\Patrick\Desktop\Nadja\Nadjas Dateien\ufo_sightings_scrubbed.csv")


In [ ]:
merged_df.columns

Index(['latitude', 'longitude', 'result_city', 'datetime', 'city', 'state',
       'country', 'shape', 'duration_seconds', 'duration_hour_min', 'comments',
       'date_posted'],
      dtype='object')

In [ ]:
# UFO Datensatz mit vervollständigten Städten
path = r"C:\Users\Admin\Desktop\07 Python für Datenanalysten\Projekt_UFO\Nadja\merged_data_ufo_sightings.csv"
merged_df = pd.read_csv(path)

merged_df.head(5)

,latitude,longitude,result_city,datetime,city,state,country,shape,duration_seconds,duration_hour_min,comments,date_posted
0,29.883056,-97.941111,san marcos,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,2004-04-27
1,29.883056,-97.941111,san marcos,2005-10-02 09:15:00,san marcos,tx,us,diamond,5400,1.5 hours,Very bright light was seen over South direction,2005-10-11
2,29.883056,-97.941111,san marcos,1965-12-10 21:00:00,san marcos,tx,us,light,60,1 minute,"What happened on that buss in 1965, in Texas",2003-10-31
3,29.883056,-97.941111,san marcos,2014-01-24 19:23:00,san marcos,tx,us,disk,720,12 minutes,I was heading toward downtown from my house on...,2014-01-30
4,29.883056,-97.941111,san marcos,2009-01-30 20:04:00,san marcos,tx,us,triangle,1500,25 minutes,"object flew over me, in triangle shape with re...",2009-03-19


In [ ]:
# Anzahl der Zeilen anzeigen
num_rows = merged_df.shape[0]
print(f"Anzahl der Zeilen: {num_rows}")

Anzahl der Zeilen: 3768838


In [ ]:
merged_df.columns

Index(['latitude', 'longitude', 'result_city', 'datetime', 'city', 'state',
       'country', 'shape', 'duration_seconds', 'duration_hour_min', 'comments',
       'date_posted'],
      dtype='object')

In [ ]:
# Häufigkeit der Sichtungsformen pro Stadt
shape_city_counts = merged_df.groupby(['result_city', 'shape']).size().reset_index(name='count')
shape_city_counts

,result_city,shape,count
0,&ccedil;anakkale (turkey),triangle,1
1,&iacute;safj&ouml;r&eth;ur (iceland),cylinder,1
2,100 mile (canada),circle,1
3,100 mile (canada),disk,1
4,a55 northope (uk/north wales),unknown,1
...,...,...,...
33128,zion national park,light,2
33129,zion national park,oval,2
33130,zlatoust (russia),sphere,1
33131,zortman,light,1


In [ ]:
# Diagramm erstellen        --> macht keinen Sinn, da zu viele Städte vohanden sind
shape_city_counts.plot(kind='bar', stacked=True, figsize=(10, 6))

# Diagramm beschriften
plt.title('UFO Sichtungen nach Form in verschiedenen Städten')
plt.xlabel('Stadt')
plt.ylabel('Anzahl der Sichtungen')
plt.legend(title='Form')
plt.xticks(rotation=45)
plt.tight_layout()

# Diagramm anzeigen
plt.show()

NameError: name 'shape_city_counts' is not defined

In [ ]:
# Häufigkeitstabelle erstellen
shape_counts = merged_df.groupby(['result_city', 'shape']).size().unstack(fill_value=0)
shape_counts

shape,changed,changing,chevron,cigar,circle,cone,crescent,cross,cylinder,delta,...,light,other,oval,pyramid,rectangle,round,sphere,teardrop,triangle,unknown
result_city,,,,,,,,,,,,,,,,,,,,,
&ccedil;anakkale (turkey),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
&iacute;safj&ouml;r&eth;ur (iceland),0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
&ouml;lmstad (sweden),0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1-25 corridor (southbound&#44 65 miles north nm border),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100 mile (canada),0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zuiddorpe (netherlands),0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
zumbrota,0,0,0,3,3,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0
zutphen (netherlands),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
# Diagramm erstellen  --> macht keinen Sinn, da zu viele Städte vohanden sind
shape_counts.plot(kind='bar', stacked=True, figsize=(10, 6))

# Diagramm beschriften
plt.title('UFO Sichtungen nach Form in verschiedenen Städten')
plt.xlabel('Stadt')
plt.ylabel('Anzahl der Sichtungen')
plt.legend(title='Form')
plt.xticks(rotation=45)
plt.tight_layout()

# Diagramm anzeigen
plt.show()

KeyboardInterrupt: 

: 

: 